In [9]:
import torch
from smoothquant.smooth import smooth_lm
from smoothquant.utils import Perplexity
from smoothquant.llama import Int8LlamaForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
from smoothquant.calibration import get_act_scales, get_static_llama_decoder_layer_scales

In [10]:
model_name = 'TheBloke/Llama-2-7b-chat-fp16'
# model_name = 'PY007/TinyLlama-1.1B-Chat-v0.2'

## FP16 Model Accuracy

In [11]:
model_fp16 = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16, 
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [12]:
ppl = Perplexity(model_fp16, tokenizer)
out = ppl.calculate_perplexity()
print(f'FP16 perplexity: {out[-1]}')

Perplexity: - :   0%|          | 0/655 [00:00<?, ?it/s]

FP16 perplexity: 7.627832972676341


## SmoothQuant W8A8 Quantized

In [13]:
# load model
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16, 
    device_map='auto',
    low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# smooth layers
act_scales = get_act_scales(model, tokenizer, 'mit-han-lab/pile-val-backup', 512, 512)
smooth_lm(model, act_scales, 0.5)

# get model scales
decoder_layer_scales, raw_scales = get_static_llama_decoder_layer_scales(
    model,
    tokenizer,
    'mit-han-lab/pile-val-backup',
    num_samples=512,
    seq_len=512
)

with torch.device("cuda"):
    model_smoothquant_w8a8 = Int8LlamaForCausalLM.from_float(model, decoder_layer_scales)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.
100%|██████████| 512/512 [01:01<00:00,  8.38it/s]


Mean input scale: 20.86: 100%|██████████| 512/512 [01:02<00:00,  8.21it/s]


In [15]:
ppl = Perplexity(model_smoothquant_w8a8, tokenizer)
out = ppl.calculate_perplexity()
print(f'SmoothQuant W8A8 perplexity: {out[-1]}')

Perplexity: - :   0%|          | 0/655 [00:00<?, ?it/s]

KeyboardInterrupt: 